In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import SCM_MEBM as scm_mebm
import matplotlib as mpl

In [ ]:
mpl.rcParams['axes.titlesize']        = 10  
mpl.rcParams['legend.fontsize']       = 10  
mpl.rcParams['legend.title_fontsize'] = 10  
mpl.rcParams['xtick.labelsize']       = 10  
mpl.rcParams['ytick.labelsize']       = 10  
mpl.rcParams['axes.labelsize']        = 10 

In [ ]:
# --- set up grid and time-stepping ---

n   = 120                           # number of grid points 
dx  = 2./n                          # width of grid box
x   = np.linspace(-1+dx/2,1-dx/2,n) # native grid
xb  = np.linspace(-1+dx,1-dx,n-1)   # boundaries of grid
nt  = 1000                          # number of timesteps
dur = 100                           # length of simulation in years
dt  = 1./nt                         # timestep in years

grid = {'n': n, 'dx': dx, 'x': x, 'xb': xb, 'nt': nt, 'dur': dur, 'dt': dt} 

Ti  = 7.5+20*(1-2*x**2)             # initial condition 

# --- run the SCM-MEBM ---

time, Es, Ts, Hi, ASR = scm_mebm.model(grid, Ti, F = 0, sea_ice_albedo = 'on', sea_ice_thermodynamics = 'on')

In [ ]:
# --- process model output ---

# extract Northern Hemisphere output
n_2   = int(n/2)+1                 # number of grid points in the Northern Hemisphere
x_n   = x[-n_2:]                   # sine of latitude
lat_n = np.rad2deg(np.arcsin(x_n)) # latitude
Ts    = Ts[-n_2:,:]                # surface temperature 
Es    = Es[-n_2:,:]                # surface enthalpy
Hi    = Hi[-n_2:,:]                # sea ice thickness

# extract winter and summer seasons
winter = np.argmin(np.mean(Ts, axis=0))
summer = np.argmax(np.mean(Ts, axis=0))

In [ ]:
fig = plt.figure(figsize=(15,3))

# plot seasonal surface temperature
plt.subplot(141)
plt.contourf(time,lat_n,Ts,np.linspace(-20,20,11),cmap='RdBu_r',extend='both')
cb = plt.colorbar()
cb.set_label('Surface temperature [ °C ]')
plt.xlabel('Time')
plt.ylabel('Latitude')
plt.ylim(0,np.max(lat_n))
plt.yticks([0,25,50,75],['0°','25°N','50°N','75°N'])

# plot seasonal sea ice thickness
plt.subplot(142)
clevsh = np.linspace(0,2,11)
plt.contourf(time,lat_n,Hi,clevsh,extend='max',cmap='Blues_r')
cb = plt.colorbar()
cb.set_label('Sea ice thickness [ m ]')
# plot ice edge on h
plt.contour(time,lat_n,Hi,[0],colors='k')
plt.xlabel('Time')
plt.ylabel('Latitude')
plt.ylim(0,np.max(lat_n))
plt.yticks([0,25,50,75],['0°','25°N','50°N','75°N'])

# plot surface temperature profiles
plt.subplot(143)
Annual, = plt.plot(lat_n,np.nanmean(Ts[:,:],axis=1),'k',label='Annual')
Summer, = plt.plot(lat_n,Ts[:,summer],'k--',label='Summer')
Winter, = plt.plot(lat_n,Ts[:,winter],'k:',label='Winter')
plt.xticks([0,25,50,75],['0°','25°N','50°N','75°N'])
plt.xlim(0,np.max(lat_n))
plt.ylim(-30,30)
plt.yticks([-30,-15,0,15,30])
plt.xlabel('Latitude')
plt.ylabel('Surface temperature [ °C ]')
plt.legend(handles = [Annual,Summer,Winter],loc=0)
plt.grid()

# plot sea ice thickness profiles
plt.subplot(144)
plt.plot(lat_n,np.nanmean(Hi[:,:],axis=1),'k',label='Annual')
plt.plot(lat_n,Hi[:,summer],'k--')
plt.plot(lat_n,Hi[:,winter],'k:')
plt.ylim([0,16])
plt.yticks([0,4,8,12,16])

plt.xticks([45,60,75,90],['45°N','60°N','75°N','90°N'])
plt.xlim(45,np.max(lat_n))
plt.xlabel('Latitude')
plt.ylabel('Sea ice thickness [ m ]')
plt.grid()

plt.rcParams.update({'font.size': 10})

plt.tight_layout() 